# a

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, roc_curve, 
                            precision_recall_curve, average_precision_score)
from sklearn.datasets import load_files
import warnings
warnings.filterwarnings('ignore')

def carregar_dados_imdb():
    dados_treino = load_files("../ft_engine/aclImdb/train", categories=['pos', 'neg'], encoding='utf-8', random_state=42)
    dados_teste = load_files("../ft_engine/aclImdb/test/", categories=['pos', 'neg'], encoding='utf-8', random_state=42)
    return dados_treino.data, dados_treino.target, dados_teste.data, dados_teste.target

X_train, y_train, X_test, y_test = carregar_dados_imdb()

def vetorizar_tfidf(textos_treino, textos_teste):
    vetor_config = {
        "min_df": 5,
        "stop_words": "english",
        "ngram_range": (1, 2)
    }
    vetor = TfidfVectorizer(**vetor_config)
    X_train_tfidf = vetor.fit_transform(textos_treino)
    X_test_tfidf = vetor.transform(textos_teste)
    print(f"Número de documentos de treino: {X_train_tfidf.shape[0]}")
    print(f"Número de features: {X_train_tfidf.shape[1]}")
    return vetor, X_train_tfidf, X_test_tfidf

vetor, X_train_tfidf, X_test_tfidf = vetorizar_tfidf(X_train, X_test)

def analisar_knn_variando_k():
    # Padronizar os dados manualmente (sem usar Pipeline)
    scaler = StandardScaler(with_mean=False)  # Para matrizes esparsas
    X_train_scaled = scaler.fit_transform(X_train_tfidf)
    X_test_scaled = scaler.transform(X_test_tfidf)
    
    # Intervalo de K de 1 a 22
    k_values = range(1, 23)
    
    # Listas para armazenar as métricas
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    aucs = []
    
    print("Analisando valores de K de 1 a 22...")
    
    for k in k_values:
        print(f"Processando K = {k}...")
        
        # Criar e treinar o modelo KNN
        knn = KNeighborsClassifier(n_neighbors=k)
        
        # Calcular métricas com validação cruzada
        accuracy = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy').mean()
        precision = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='precision').mean()
        recall = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='recall').mean()
        f1 = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='f1').mean()
        
        # Treinar modelo completo para calcular AUC
        knn.fit(X_train_scaled, y_train)
        y_prob = knn.predict_proba(X_train_scaled)[:, 1]
        auc = roc_auc_score(y_train, y_prob)
        
        # Armazenar métricas
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        aucs.append(auc)
    
    # Plotar todas as métricas
    plt.figure(figsize=(14, 10))
    
    # Acurácia
    plt.subplot(2, 2, 1)
    plt.plot(k_values, accuracies, 'bo-', linewidth=2)
    plt.xlabel('Valor de K')
    plt.ylabel('Acurácia Média (Validação Cruzada)')
    plt.title('Acurácia vs. Valor de K')
    plt.grid(True)
    plt.xticks(k_values, rotation=45)
    
    # Precisão e Recall
    plt.subplot(2, 2, 2)
    plt.plot(k_values, precisions, 'go-', label='Precisão', linewidth=2)
    plt.plot(k_values, recalls, 'ro-', label='Recall', linewidth=2)
    plt.xlabel('Valor de K')
    plt.ylabel('Pontuação Média (Validação Cruzada)')
    plt.title('Precisão e Recall vs. Valor de K')
    plt.legend()
    plt.grid(True)
    plt.xticks(k_values, rotation=45)
    
    # F1-Score
    plt.subplot(2, 2, 3)
    plt.plot(k_values, f1_scores, 'mo-', linewidth=2)
    plt.xlabel('Valor de K')
    plt.ylabel('F1-Score Médio (Validação Cruzada)')
    plt.title('F1-Score vs. Valor de K')
    plt.grid(True)
    plt.xticks(k_values, rotation=45)
    
    # AUC
    plt.subplot(2, 2, 4)
    plt.plot(k_values, aucs, 'co-', linewidth=2)
    plt.xlabel('Valor de K')
    plt.ylabel('AUC Média')
    plt.title('AUC (ROC) vs. Valor de K')
    plt.grid(True)
    plt.xticks(k_values, rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Encontrar o melhor K baseado no F1-Score (trade-off entre precisão e recall)
    melhor_k_idx = np.argmax(f1_scores)
    melhor_k = k_values[melhor_k_idx]
    
    print(f"\nMelhor valor de K: {melhor_k} (F1-Score máximo: {f1_scores[melhor_k_idx]:.4f})")
    
    # Plotar curva ROC para o melhor K
    plotar_curva_roc(melhor_k, X_train_scaled, X_test_scaled, y_train, y_test)
    
    return melhor_k

def plotar_curva_roc(k, X_train_scaled, X_test_scaled, y_train, y_test):
    # Treinar modelo com o melhor K
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    # Obter probabilidades para a classe positiva
    y_prob = knn.predict_proba(X_test_scaled)[:, 1]
    
    # Calcular FPR, TPR e AUC
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    
    # Plotar curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'KNN (K={k}, AUC = {auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos (FPR)')
    plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
    plt.title('Curva ROC para o Melhor K')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()
    
    # Calcular e exibir métricas detalhadas para o melhor K
    y_pred = knn.predict(X_test_scaled)
    print("\nMétricas detalhadas para o melhor K:")
    print(f"📌 Precisão: {precision_score(y_test, y_pred):.4f}")
    print(f"📌 Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"📌 F1-Score: {f1_score(y_test, y_pred):.4f}")
    print(f"📌 AUC (ROC): {auc:.4f}")
    
    # Plotar curva Precision-Recall
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    avg_precision = average_precision_score(y_test, y_prob)
    
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2,
             label=f'KNN (K={k}, AP={avg_precision:.4f})')
    plt.xlabel('Recall')
    plt.ylabel('Precisão')
    plt.title('Curva Precision-Recall')
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.show()

# Executar a análise
melhor_k = analisar_knn_variando_k()

Número de documentos de treino: 20476
Número de features: 66970
Analisando valores de K de 1 a 22...
Processando K = 1...
Processando K = 2...
Processando K = 3...
Processando K = 4...
Processando K = 5...
Processando K = 6...
Processando K = 7...
Processando K = 8...
Processando K = 9...


KeyboardInterrupt: 